# Synopsis
Add vector space model to database

# Configuration

In [1]:
db_name = 'sitcoms.db'
OHCO = ['show_num', 'seas_num', 'epis_num', 'sent_num', 'token_num']

# Libraries

In [2]:
import sqlite3
import pandas as pd
import numpy as np

# Process

In [3]:
with sqlite3.connect(db_name) as db:
    K = pd.read_sql('SELECT * FROM token', db, index_col=OHCO)
    V = pd.read_sql('SELECT * FROM vocab', db, index_col='term_id')

## Create DTM


### Create word mask

Filter out stopwords and proper nouns

In [4]:
WORDS = (K.punc == 0) & (K.num == 0) & (K.pos != 'NNP') & (K.pos != 'NNPS') & K.term_id.isin(V[V.stop==0].index)

### Extract BOW from tokens

To extract a bag-of-words model from our tokens table, we apply a simple `groupby()` operation.

In [5]:
BOW = K[WORDS].groupby(OHCO[:3]+['term_id'])['term_id'].count()

### Convert BOW to DTM

In [6]:
DTM = BOW.unstack().fillna(0)

In [7]:
#DTM

## Compute Term Frequencies and Weights

### Compute TF

In [8]:
alpha = .000001 # We introduce an arbitrary smoothing value
alpha_sum = alpha * V.shape[0]
TF = DTM.apply(lambda x: (x + alpha) / (x.sum() + alpha_sum), axis=1)

### Compute TFIDF

In [9]:
N_docs = DTM.shape[0]
V['df'] = DTM[DTM > 0].count()
TFIDF = TF * np.log2(N_docs / V[V.stop==0]['df'])

### Compute TFTH (Experiment)

In [10]:
#THM = -(TF * np.log2(TF))
#TFTH = TF.apply(lambda x: x * THM.sum(), 1)

### Add stats to V

In [11]:
#V['tf_sum'] = TF.sum()
V['tf_mean'] = TF.mean()
V['tf_max'] = TF.max()
#V['tfidf_sum'] = TFIDF.sum()
V['tfidf_mean'] = TFIDF.mean()
V['tfidf_max'] = TFIDF.max()
#V['tfth_sum'] = TFTH.sum()
#V['tfth_mean'] = TFTH.mean()
#V['tfth_max'] = TFTH.max()
#V['th_sum'] = THM.sum()
#V['th_mean'] = THM.mean()
#V['th_max'] = THM.max()

## Create Docs table

In [12]:
D = DTM.sum(1).astype('int').to_frame().rename(columns={0:'term_count'})
D['tf'] = D.term_count / D.term_count.sum()
f = open('episodes.txt', 'r') # add episode titles
l = f.readlines()
D['title'] = l
D.title = D.title.str.strip()

In [13]:
D.title

show_num  seas_num  epis_num
0         0         0                                          Pilot (The Office)
                    1                                               Diversity Day
                    2                                                 Health Care
                    3                                                The Alliance
                    4                                                  Basketball
                    5                                                    Hot Girl
          1         0                                                 The Dundies
                    1                                           Sexual Harassment
                    2                                             Office Olympics
                    3                                                    The Fire
                    4                                                   Halloween
                    5                                      The Fight 

## Fast Way

## Get all doc pairs

In [14]:
epis_ids = D.index.tolist()
pairs = [(i,j) for i in epis_ids for j in epis_ids if j != i]
P = pd.DataFrame(pairs).reset_index(drop=True).set_index([0,1])
P.index.names = ['doc_x','doc_y']

## Compute Euclidean distance

In [15]:
def euclidean(row):
    D1 = TFIDF.loc[row.name[0]]
    D2 = TFIDF.loc[row.name[1]]
    x = (D1 - D2)**2
    y = x.sum() 
    z = np.sqrt(y)
    return z

In [16]:
#P['euclidean'] = 0
#P['euclidean'] = P.apply(euclidean, 1)

## Compute Cosine similarity

In [17]:
def cosine(row):
    D1 = TFIDF.loc[row.name[0]]
    D2 = TFIDF.loc[row.name[1]]
    x = D1 * D2
    y = x.sum()
    a = np.sqrt((D1**2).sum())
    b = np.sqrt((D2**2).sum())
    c = a * b
    z = y / c
    return z

In [18]:
P['cosine'] = 0
P['cosine'] = P.apply(cosine, 1)

In [19]:
P = P.reset_index() # sqlite can't save tuples, so converted to strings
P[['doc_x', 'doc_y']] = P[['doc_x', 'doc_y']].astype(str)

In [20]:
#P

# Create small TFIDF table

In [21]:
def get_top_terms(vocab, no_stops=True, sort_col='n', k=1000):
    if no_stops:
        V = vocab[vocab.stop == 0]
    else:
        V = vocab
    return V.sort_values(sort_col, ascending=False).head(k)

In [22]:
proper_nouns = K.loc[(K.pos == 'NNP') & (K.pos == 'NNPS'), 'term_id'].unique()
TOPV = get_top_terms(V.loc[~V.index.isin(proper_nouns)])

In [23]:
TOPV

,term_str,n,p,port_stem,stop,df,tf_mean,tf_max,tfidf_mean,tfidf_max
term_id,,,,,,,,,,
120,'s,20226,0.019204,'s,0,295.0,0.043636,0.080778,0.002715,0.005026
14878,n't,10844,0.010296,n't,0,295.0,0.023536,0.042906,0.001464,0.002669
82,'m,7537,0.007156,'m,0,293.0,0.016205,0.029869,0.001167,0.002151
15549,oh,6326,0.006006,oh,0,308.0,0.013222,0.028199,0.000000,0.000000
12522,know,6325,0.006005,know,0,308.0,0.013739,0.029084,0.000000,0.000000
115,'re,5846,0.005551,'re,0,293.0,0.012454,0.024282,0.000897,0.001749
13058,like,5254,0.004988,like,0,308.0,0.011387,0.026885,0.000000,0.000000
15579,okay,4884,0.004637,okay,0,258.0,0.002832,0.013628,0.000724,0.003483
25556,yeah,4642,0.004407,yeah,0,308.0,0.010135,0.023371,0.000000,0.000000


# Save

In [24]:
with sqlite3.connect(db_name) as db:
    V.to_sql('vocab', db, if_exists='replace', index=True)
    #K.to_sql('token', db, if_exists='replace', index=True)
    D.to_sql('doc', db, if_exists='replace', index=True)
    P.to_sql('docpair', db, if_exists='replace', index=False)
    #BOW.to_frame().rename(columns={'term_id':'n'}).to_sql('bow', db, if_exists='replace', index=True)
    TFIDF[TOPV.index].stack().to_frame().rename(columns={0:'term_weight'})\
    .to_sql('tfidf_small', db, if_exists='replace', index=True)

# End